In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import date, datetime, timedelta
import json
import pandas as pd
import time
import os
from pathlib import Path
import FlightMod.CrawlList as fcl

In [2]:
## 函式庫

# def safe_extract(func):
#     """判斷一個soup物件是否存在/有值，若沒有則回傳None"""
#     try:
#         return func()
#     except (IndexError, AttributeError):
#         return None


# def split_airport_code(code):
#     if code is not None:
#         code = code.replace("(", "").replace(")", "")
#         if "/" in code:
#             code1, code2 = code.split("/")
#             code1 = code1.strip()
#             code2 = code2.strip()
#         else:
#             code1 = code
#             code2 = code
#     else:
#         code1 = None
#         code2 = None
    
#     return code1, code2

In [3]:
# 判斷總列表.csv檔是否存在，若不存在則先建立一個只有columns的空表格
file = 'C:/Users/add41/Documents/Data_Engineer/docker/Develop/FlightCrawler/FlightData/EVA_FlightList.csv'
list_path = Path(file)

if list_path.exists():
    pass
else:
    columns = [
        'query_date',
        'flight_no',
        'flight_type',
        'departure_airport',
        'departure_airport_code_1',
        'departure_airport_code_2',
        'arrival_airport',
        'arrival_airport_code_1',
        'arrival_airport_code_2',
        'link',
        'sync'
    ]

    df = pd.DataFrame(columns=columns)
    df.to_csv(file, index=False)


In [4]:
# 讀入總表檔案
df_list = pd.read_csv(file)

In [5]:
# 建立空list（為建立dataframe預備）並設定起始頁數，建立ss連線
data = []
logtime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

start_page = 0
ss = requests.Session()


while True:
    # 網頁為20筆一頁，設定從第0筆開始查詢，每次回圈+20，直到查無資料後break
    # 利用ss.get發出請求並轉換出soup物件
    url = f'https://www.flightaware.com/live/fleet/EVA?;offset={start_page};order=ident;sort=ASC'
    headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36'}
    res = ss.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    print(f'開始查詢{start_page}到{start_page + 20}筆資料...')


    # 先判斷是否有資料，若找到「查無資料」的標籤，則設 found = True 終止迴圈
    found = False
    for tag in soup.find_all('i'):
        if tag.text == "Sorry. No matching flights found; try again later.":
            found = True
            break


    # 若仍有資料 found = False 則繼續迴圈，分別尋找兩個標籤（兩種皆有連結）並合併list
    if found == False:
        table_list = soup('table', class_='prettyTable fullWidth')[0]('tr')[2:]

        for i in table_list:
            single = []
            flight_no = i('td')[0].span.a.text
            print(f'查詢{flight_no}班機資料...')

            # 紀錄日期
            single.append(logtime)

            # 班機編號
            single.append(fcl.safe_extract(lambda: i('td')[0].span.a.text))

            # 機型
            single.append(fcl.safe_extract(lambda: i('td')[1].span.a.text))

            # 起飛機場
            single.append(fcl.safe_extract(lambda: i('td')[2]('span', dir='ltr')[0].text))

            # 起飛機場代號（如有兩種則分開儲存，只有一種則重複儲存）
            code_d = fcl.safe_extract(lambda: i('td')[2]('span', dir='ltr')[1].text)
            code_d1, code_d2 = fcl.split_airport_code(code_d)
            single.append(code_d1)
            single.append(code_d2)


            # 降落機場
            single.append(fcl.safe_extract(lambda: i('td')[3]('span', dir='ltr')[0].text))

            # 降落機場代號（如有兩種則分開儲存，只有一種則重複儲存）
            code_a = fcl.safe_extract(lambda: i('td')[3]('span', dir='ltr')[1].text)
            code_a1, code_a2 = fcl.split_airport_code(code_a)
            single.append(code_a1)
            single.append(code_a2)

            # 連結
            single.append('https://www.flightaware.com' + fcl.safe_extract(lambda: i('td')[0].span.a['href']))

            # 同步標記
            single.append(0)

            data.append(single)
            print(f'完成{flight_no}班機資料存取')


        # 完成後查詢筆數+20並稍微等待後在進行下一次迴圈
        print(f'完成存取{start_page}到{start_page + 20}筆資料')
        start_page += 20
        time.sleep(15)


    # 當查無資料時 found = True 顯示查無資料並終止迴圈
    else:
        print(f'沒有{start_page}到{start_page + 20}筆資料')
        break

print(f'已完成存取資料')

開始查詢0到20筆資料...
查詢EVA105班機資料...
完成EVA105班機資料存取
查詢EVA107班機資料...
完成EVA107班機資料存取
查詢EVA109班機資料...
完成EVA109班機資料存取
查詢EVA116班機資料...
完成EVA116班機資料存取
查詢EVA118班機資料...
完成EVA118班機資料存取
查詢EVA157班機資料...
完成EVA157班機資料存取
查詢EVA165班機資料...
完成EVA165班機資料存取
查詢EVA169班機資料...
完成EVA169班機資料存取
查詢EVA177班機資料...
完成EVA177班機資料存取
查詢EVA181班機資料...
完成EVA181班機資料存取
查詢EVA189班機資料...
完成EVA189班機資料存取
查詢EVA191班機資料...
完成EVA191班機資料存取
查詢EVA201班機資料...
完成EVA201班機資料存取
查詢EVA215班機資料...
完成EVA215班機資料存取
查詢EVA227班機資料...
完成EVA227班機資料存取
查詢EVA234班機資料...
完成EVA234班機資料存取
查詢EVA237班機資料...
完成EVA237班機資料存取
查詢EVA255班機資料...
完成EVA255班機資料存取
查詢EVA282班機資料...
完成EVA282班機資料存取
查詢EVA315班機資料...
完成EVA315班機資料存取
完成存取0到20筆資料
開始查詢20到40筆資料...
查詢EVA391班機資料...
完成EVA391班機資料存取
查詢EVA49班機資料...
完成EVA49班機資料存取
查詢EVA5班機資料...
完成EVA5班機資料存取
查詢EVA501班機資料...
完成EVA501班機資料存取
查詢EVA6班機資料...
完成EVA6班機資料存取
查詢EVA62班機資料...
完成EVA62班機資料存取
查詢EVA630班機資料...
完成EVA630班機資料存取
查詢EVA633班機資料...
完成EVA633班機資料存取
查詢EVA646班機資料...
完成EVA646班機資料存取
查詢EVA65班機資料...
完成EVA65班機資料存取
查詢EVA651班機資料...
完成EVA651班機資料存取
查詢EVA668班機

In [6]:
# 根據爬蟲資料建立Dataframe
columns = [
    'query_date',
    'flight_no',
    'flight_type',
    'departure_airport',
    'departure_airport_code_1',
    'departure_airport_code_2',
    'arrival_airport',
    'arrival_airport_code_1',
    'arrival_airport_code_2',
    'link',
    'sync'
]

df2 = pd.DataFrame(columns=columns, data=data)
print('新資料建檔完成')

新資料建檔完成


In [7]:
# 將機場名稱中的Int'l字樣去除，並去除前後空白
df2['departure_airport'] = df2['departure_airport'].str.replace("Int\'l", "").str.replace("Intl", "").str.strip()
df2['arrival_airport'] = df2['arrival_airport'].str.replace("Int\'l", "").str.replace("Intl", "").str.strip()

In [8]:
# 直接將新資料與舊資料合併
df_combine = pd.concat([df_list, df2], ignore_index=True)

# 對合併後的資料使用drop_duplicates，將重複值刪去，並覆蓋回df_list
df_list = df_combine.drop_duplicates(subset='link', keep = 'first').reset_index(drop=True)

# 將query_date欄位轉換為datetime物件
df_list['query_date'] = pd.to_datetime(df_list['query_date'])

# 將新的df_list進行存檔
df_list.to_csv(file, index=False)

print('完成資料更新')

完成資料更新


In [12]:
len(df_list)

195